In [33]:
# Importações necessárias
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
DLL load failed while importing _pywrap_tf2: Uma rotina de inicialização da biblioteca de vínculo dinâmico (DLL) falhou.

In [25]:
df = pd.read_csv("Dmoz-sports.csv")
df = df[['text', 'class']]

In [26]:
df.groupby('class').count()

,text
class,
Baseball,500
Basketball,500
Bowling,500
Cricket,500
Cycling,500
Equestrian,500
Fencing,500
Flying,500
Football,500


In [27]:
# 2. Dividir os dados em treino (70%), validação (10%) e teste (20%)
temp_texts, test_texts, temp_labels, test_labels = train_test_split(
    df['text'], df['class'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.125, random_state=42)  # 10% validação (0.125 * 0.8 = 0.1 total)

In [28]:
# 3. Tokenização com BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [29]:
# Tokenizar os textos
def tokenize_texts(texts):
    return tokenizer(list(texts), truncation=True, padding=True, max_length=512, return_tensors="pt")

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)
test_encodings = tokenize_texts(test_texts)

In [30]:
# Criar um mapeamento das classes para números
label_map = {label: idx for idx, label in enumerate(df['class'].unique())}
train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)
test_labels = test_labels.map(label_map)

# Converter os labels para tensores
train_labels = torch.tensor(train_labels.values)
val_labels = torch.tensor(val_labels.values)
test_labels = torch.tensor(test_labels.values)

print("Pré-processamento concluído!")

Pré-processamento concluído!


In [31]:
# Criar Dataset personalizado
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

NameError: name 'Dataset' is not defined

In [ ]:
# Criar DataLoaders
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Configurar otimizador e função de perda
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Loop de treinamento
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

print("Treinamento concluído!")
